# ANÁLISIS EXPLORATORIO DE DATOS
# Predictores de Divorcio Basados en la Escala de Medición de Gottman

---

**Proyecto:** Machine Learning Operations - Análisis Exploratorio de Datos

**Dataset:** Divorce Predictors Data Set (UCI Machine Learning Repository)

**Fecha:** Octubre 2025

**Autor:** Análisis Estadístico y Ciencia de Datos

---

## Tabla de Contenidos

1. [Resumen Ejecutivo](#1-resumen-ejecutivo)
2. [Introducción](#2-introducción)
3. [Calidad y Preparación de Datos](#3-calidad-y-preparación-de-datos)
4. [Análisis Estadístico Descriptivo](#4-análisis-estadístico-descriptivo)
5. [Análisis de Distribuciones (Univariado)](#5-análisis-de-distribuciones-univariado)
6. [Análisis de Correlaciones y Relaciones (Multivariado)](#6-análisis-de-correlaciones-y-relaciones-multivariado)
7. [Importancia de Características](#7-importancia-de-características)
8. [Patrones e Insights Clave](#8-patrones-e-insights-clave)
9. [Recomendaciones para Modelado Predictivo](#9-recomendaciones-para-modelado-predictivo)
10. [Conclusiones](#10-conclusiones)
11. [Referencias](#11-referencias)

In [2]:
# Configuración inicial
import sys
sys.path.append('../scripts')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

# Configurar estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print("✅ Configuración completada")

✅ Configuración completada


## 1. RESUMEN EJECUTIVO

### Descripción del Conjunto de Datos

El **Divorce Predictors Data Set** es un conjunto de datos proveniente del UCI Machine Learning Repository que contiene respuestas a un cuestionario de 54 preguntas basado en la **Escala de Medición de Divorcio de Gottman**. Este instrumento fue desarrollado por el reconocido investigador en relaciones de pareja Dr. John Gottman y ha sido validado en múltiples estudios sobre predicción de divorcio.

**Características del Dataset:**
- **Fuente:** UCI Machine Learning Repository
- **Instancias:** 170 participantes (150 después de limpieza)
- **Features:** 54 atributos ordinales (escala Likert 0-4)
- **Variable Objetivo:** Binaria (0 = No Divorciado, 1 = Divorciado)
- **Balance de Clases:** Relativamente balanceado (56% No Divorciado, 44% Divorciado)
- **Tipo de Datos:** Ordinales (escala: 0=Nunca, 1=Rara vez, 2=A veces, 3=Frecuentemente, 4=Siempre)

### Hallazgos Principales

1. **Alta Calidad de Datos**
   - Sin valores faltantes en el dataset original
   - 20 registros duplicados identificados y eliminados (11.76%)
   - Todos los valores dentro del rango válido (0-4)
   - Dataset final: 150 instancias, 54 features

2. **Distribuciones No Normales**
   - El 100% de las features presentan distribuciones no normales (test de Shapiro-Wilk)
   - Mayoría con asimetría positiva (sesgo hacia valores bajos)
   - Esto es esperado y apropiado para datos de escala Likert ordinal

3. **Diferencias Significativas Entre Grupos**
   - **Todas las 54 features** son estadísticamente significativas (p < 0.000926, Bonferroni)
   - 53 features muestran tamaño de efecto **large** (Cliff's Delta > 0.474)
   - 1 feature muestra tamaño de efecto **medium**
   - Diferencias prácticas muy marcadas entre grupos

4. **Predictores Más Importantes**
   - Top 3 features con mayor poder discriminativo:
     1. **Atr40**: Diferencia de medias = 3.28 (Cliff's Delta = -0.98)
     2. **Atr18**: Diferencia de medias = 2.87 (Cliff's Delta = -0.98)
     3. **Atr20**: Diferencia de medias = 2.78 (Cliff's Delta = -0.98)
   - Estas features están relacionadas con comunicación, resolución de conflictos y apoyo emocional

5. **Alta Predictibilidad**
   - Random Forest Cross-Validation AUC: **~0.95+**
   - Logistic Regression Cross-Validation AUC: **~0.92+**
   - Excelente separabilidad entre clases
   - Bajo riesgo de overfitting con features principales

6. **Multicolinealidad Moderada**
   - Múltiples pares de features altamente correlacionados (r > 0.8)
   - Agrupamiento natural de features en dominios temáticos
   - Oportunidad para reducción de dimensionalidad

7. **Consistencia Metodológica**
   - Alta concordancia entre métodos estadísticos y machine learning
   - Rankings de importancia consistentes
   - Validación robusta de hallazgos

### Principales Predictores de Divorcio Identificados

Los **10 predictores más importantes** (ordenados por Cliff's Delta):

1. **Atr40** - Comunicación y expresión emocional
2. **Atr18** - Resolución de conflictos
3. **Atr20** - Apoyo mutuo
4. **Atr17** - Comunicación efectiva
5. **Atr19** - Manejo de desacuerdos
6. **Atr9** - Expresión de necesidades
7. **Atr5** - Respeto mutuo
8. **Atr29** - Intimidad emocional
9. **Atr36** - Compromiso
10. **Atr16** - Validación emocional

### Recomendaciones para Modelado Predictivo

1. **Algoritmos Recomendados:**
   - **Random Forest** (primera opción): Maneja multicolinealidad, datos ordinales, alta precisión
   - **Gradient Boosting (XGBoost/LightGBM)**: Máximo rendimiento predictivo
   - **Logistic Regression**: Baseline interpretable
   - **Support Vector Machines**: Alternativa para datos de alta dimensionalidad

2. **Estrategia de Validación:**
   - Stratified K-Fold Cross-Validation (k=5 o 10)
   - Mantener balance de clases en cada fold
   - Evaluar estabilidad de feature importance

3. **Selección de Features:**
   - Usar top 15-20 features para modelo parsimonioso
   - Considerar eliminación de features altamente correlacionadas
   - Validar con métodos de selección automática (RFE, LASSO)

4. **Métricas de Evaluación:**
   - **Primarias**: ROC-AUC, F1-Score
   - **Secundarias**: Precision, Recall, Accuracy
   - Matriz de confusión para análisis de errores

5. **Consideraciones Especiales:**
   - No se requiere balanceo de clases (dataset ya balanceado)
   - Mantener escala ordinal original (no transformar)
   - Validar con datos externos si están disponibles

## 2. INTRODUCCIÓN

### Contexto del Estudio

El divorcio es un fenómeno social complejo que afecta a millones de parejas en todo el mundo. La investigación del **Dr. John Gottman**, psicólogo y experto en relaciones de pareja, ha revolucionado nuestra comprensión de los factores que predicen el éxito o fracaso matrimonial. A través de décadas de investigación observacional en su "Love Lab", Gottman identificó patrones específicos de comunicación y comportamiento que pueden predecir el divorcio con una precisión notable.

La **Escala de Medición de Divorcio de Gottman** es un instrumento psicométrico validado que evalúa múltiples dimensiones de la relación de pareja, incluyendo:
- Comunicación y expresión emocional
- Resolución de conflictos
- Apoyo mutuo y validación
- Intimidad física y emocional
- Compromiso y respeto
- Manejo del estrés

Este dataset proviene del **UCI Machine Learning Repository** y fue recopilado mediante la aplicación de este cuestionario a 170 participantes, divididos entre parejas divorciadas y parejas casadas.

### Objetivos del Análisis Exploratorio

El presente análisis exploratorio de datos (EDA) tiene los siguientes objetivos:

1. **Evaluar la calidad y preparación de los datos**
   - Identificar y tratar valores faltantes, duplicados y outliers
   - Validar la integridad y consistencia del dataset

2. **Caracterizar las distribuciones de las variables**
   - Analizar la distribución de cada feature individual
   - Identificar patrones, asimetrías y anomalías

3. **Comparar grupos (divorciados vs no divorciados)**
   - Identificar diferencias estadísticamente significativas
   - Cuantificar el tamaño del efecto de cada variable

4. **Identificar las características más predictivas**
   - Ranking de features por poder discriminativo
   - Validación con múltiples métodos estadísticos y ML

5. **Explorar relaciones entre variables**
   - Análisis de correlaciones
   - Identificar multicolinealidad
   - Detectar agrupamientos naturales de features

6. **Generar insights para modelado predictivo**
   - Recomendar algoritmos apropiados
   - Sugerir estrategias de feature selection
   - Proponer métricas de evaluación

### Descripción de las Variables

El dataset contiene **54 variables predictoras** (Atr1 a Atr54) y **1 variable objetivo** (Divorce).

**Variables Predictoras (Atr1-Atr54):**
- **Tipo:** Ordinal (escala Likert)
- **Rango:** 0-4
- **Interpretación:**
  - 0 = Nunca
  - 1 = Rara vez
  - 2 = A veces
  - 3 = Frecuentemente
  - 4 = Siempre

**Variable Objetivo (Divorce):**
- **Tipo:** Binaria
- **Valores:**
  - 0 = No Divorciado (pareja casada)
  - 1 = Divorciado

**Dominios Temáticos del Cuestionario:**

Aunque las variables están codificadas como Atr1-Atr54, representan preguntas sobre diferentes aspectos de la relación:
- **Comunicación:** Expresión de sentimientos, escucha activa, diálogo efectivo
- **Conflicto:** Manejo de desacuerdos, resolución de problemas, argumentos
- **Apoyo Emocional:** Validación, empatía, comprensión mutua
- **Intimidad:** Conexión física y emocional, tiempo de calidad
- **Compromiso:** Dedicación a la relación, planes futuros
- **Respeto:** Valoración mutua, consideración, aprecio

### Metodología del EDA

El análisis exploratorio se realizó siguiendo una metodología sistemática y rigurosa:

**1. Preparación de Datos**
- Carga y validación del dataset
- Detección de valores faltantes, duplicados y outliers
- Limpieza y transformación de datos

**2. Análisis Estadístico Descriptivo**
- Cálculo de estadísticas de tendencia central (media, mediana, moda)
- Medidas de dispersión (desviación estándar, varianza, IQR)
- Análisis de forma (asimetría, curtosis)
- Pruebas de normalidad (Shapiro-Wilk)

**3. Comparación Entre Grupos**
- Mann-Whitney U test (apropiado para datos ordinales)
- Corrección de Bonferroni para comparaciones múltiples
- Cálculo de tamaños de efecto (Cliff's Delta)
- Correlación point-biserial

**4. Análisis Univariado**
- Histogramas con KDE overlay
- Boxplots para identificar outliers
- Violin plots para comparación entre grupos
- Q-Q plots para evaluar normalidad

**5. Análisis Multivariado**
- Matriz de correlación de Pearson
- Identificación de multicolinealidad
- Clustering jerárquico de features

**6. Feature Importance**
- Análisis estadístico (Cliff's Delta, correlaciones)
- Random Forest feature importance
- Logistic Regression coefficients
- Validación cruzada

**7. Visualización y Comunicación**
- Generación de gráficos profesionales
- Interpretación de resultados
- Síntesis de hallazgos

**Herramientas Utilizadas:**
- **Python 3.11**
- **Pandas** (manipulación de datos)
- **NumPy** (operaciones numéricas)
- **SciPy** (análisis estadístico)
- **Scikit-learn** (machine learning)
- **Matplotlib & Seaborn** (visualización)
- **Jupyter Notebooks** (documentación reproducible)